<span style="display: inline-block; margin-right: 10px; vertical-align: middle; border: 1px solid #F8F8F8; border-radius: 8px; padding: 5px; background-color: #F8F8F8;">
    <a href="https://www.designsafe-ci.org/workspace/jupyter-lab-hpc-cuda-ds?appVersion=1.1.1" target="_parent" style="text-decoration: none;">
        <span style="font-family: Helvetica, sans-serif; font-size: 13px; color: #565656; margin-right: 5px; vertical-align: middle; font-weight: 600;">Open in</span>
        <img src="https://www.designsafe-ci.org/media/filer_public/2d/d3/2dd37fbf-289e-49cf-9c1a-879c864c4e17/nsf_nheri-ds.png" alt="Open in DesignSafe" style="width: 100px; height: auto; vertical-align: middle;">
    </a>
</span>

<span style="display: inline-block; margin-right: 10px; vertical-align: middle;">
    <a href="https://colab.research.google.com/github/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/foundation_type_classifier.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="width: 130px; height: auto; vertical-align: middle;"/>
    </a>
</span>

<span style="display: inline-block; vertical-align: middle; margin-top: 6px;">
    <a target="_blank" href="https://lightning.ai/new?repo_url=https%3A//github.com/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/foundation_type_classifier.ipynb">
        <img src="https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/app-2/studio-badge.svg" alt="Open in Studio" style="width: 120px; height: auto; vertical-align: middle;"/>
    </a>
</span>

## **Install BRAILS++**
BRAILS++ is not yet available on PyPi. The following line installs the latest version from the GitHub repository using `pip`.

In [ ]:
!pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus

## **Import ImageSet and Importer Methods of BRAILS++**
BRAILS++ offers various Artificial Intelligence-powered modules for predicting building and infrastructure attributes from images, such as the `FoundationElevationClassifier` demonstrated in this example.

The `Importer` class provides the main methods to load these modules. To gather the images needed for attribute prediction, users can use the ImageSet class or BRAILS++ scrapers, which facilitate automated asset geometry and image retrieval. This example illustrates both methods for sourcing imagery for use with the `FoundationElevationClassifier` module.

In [ ]:
from brails import Importer
from brails.types.image_set import ImageSet

## **Define Google API Key**
You need a Google API Key with Street View Static API enabled to download the Google Street View imagery required for this example. Please follow this [link](https://support.google.com/googleapi/answer/6158862?hl=en) for instructions on setting up a Google API key.

In [ ]:
API_KEY = 'YOUR-API-KEY-HERE'

## **Load Street-Level Imagery: (Option 1) Download Imagery by Specifying the Region Name**
The `FoundationElevationClassifier` class identifies if a building is elevated or not from street-level imagery. In this option, BRAILS++ retrieves the required imagery by following the steps below.
1. <strong>Call the `RegionBoundary` class with the necessary details to specify the region of interest.</strong> In this example, street-level imagery of buildings in Fort Myers, FL, are downloaded. Therefore, the required information includes the `type` of information provided and the `data` itself, which are the `locationName` and `Fort Myers, FL`.

2. <strong>Download the footprint inventory for the `RegionBoundary` saved in `region_boundary_object` by downloading the OpenStreetMap footprint data through the `OSM_FootprintScraper`.</strong> The only information required to initialize `OSM_FootprintScraper` is the desired output unit for `length`, which, in this case, is `ft`. The output of `OSM_FootprintScraper` is an `AssetInventory` saved in `ft_myers_fp`.

3. <strong>Get a randomly selected `50` building subset of the footprint inventory obtained using `OSM_FootprintScraper`. </strong> This subset is obtained using the `get_random_sample` method of an `AssetInventory` object and is stored in `ft_myers_fp_subset`. For this example, the random seed is set to `5` for reproducibility of the results.

4. <strong>Get the street-level imagery for the selected subset using `get_images` method of `GoogleStreetview` module.</strong> `get_images` requires two inputs 1) AssetInventory for which the images will be retrieved (in this example, `ft_myers_fp_subset`) and 2) the path to the folder where the retrieved images will be saved, which in this case is `tmp/street/`.

In [ ]:
# Select a region and create its RegionBoundary:
importer = Importer()

region_data = {"type": "locationName",
               "data": "Fort Myers Beach, FL"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

osm_fp_class = importer.get_class("OSM_FootprintScraper")
osm_fp_data = {"length": "ft"}
footprint_scraper = osm_fp_class(osm_fp_data)
ft_myers_fp = footprint_scraper.get_footprints(region_boundary_object)

# Subsample from the extracted assets to keep the image downloading step quick.
# Here, we are randomly sampling 50 buildings using a random seed value of 5:
ft_myers_fp_subset = ft_myers_fp.get_random_sample(50, 5)

# Get aerial imagery for the selected subset using GoogleSatellite:
google_street_class = importer.get_class("GoogleStreetview")
google_street = google_street_class({'apiKey': API_KEY})
images_street = google_street.get_images(
    ft_myers_fp_subset, "tmp/street/")

## **Predict If Building Foundations Are Elevated Using FoundationElevationClassifier Module**
`FoundationElevationClassifier` includes a pre-trained ResNet-50 model that has been trained on a custom dataset of 8,000 images. This model is capable of classifying street-level imagery of buildings into two classes: `Elevated`, and `Non-elevated`.

In [ ]:
importer = Importer()
found_elev_classifier_class = importer.get_class('FoundationElevationClassifier')
found_elev_classifier_class = found_elev_classifier_class()
predictions = found_elev_classifier_class.predict(images_street)

## **Show Prediction Results**

In [ ]:
print(predictions)